<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>CS 144 - Spring 2022 - Mini-Mapping Tool</h1></center>
<center><h1>Due: Sunday, June 5th, 2022 @ 11:59pm</h1></center>

### Enter your information below:

<div style="color: #000000;background-color: #EEEEFF">
    Your Name (submitter): Harris Shepard <br>
    Your student ID (submitter): 862132345
    
<br>
<br>
<b>By submitting this notebook, I assert that the work below is my own work, completed for this course.  Except where explicitly cited, none of the portions of this notebook are duplicated from anyone else's work or my own previous work.</b>
<br>    
<br>
<b>Instruction for submissions:</B> when you have completed this project, download this .ipynb file to your computer by left-clicking on the file name, and submit to <a href="https://elearn.ucr.edu/">Canvas</A> by the deadline. 
<br>
<br>
<B>Late work:</B> There is no late deadline for the final project, except for the most serious circumstances (illness, medical emergency, etc.) which have to be documented.
</div>


<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>Generator</h1></center>
<br>

In the first part of this project, you will write a <B>read generator</B>, that takes in input 
<UL>
<LI>a reference genome $R$ in <A HREF="https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Web&PAGE_TYPE=BlastDocs&DOC_TYPE=BlastHelp">FASTA</A> format (for instance <A HREF="http://www.cs.ucr.edu/~stelo/cs144spring22/data/BD.fa">the BD genome</A>)</LI>
    <LI>integer parameter $N$ </LI>
    <li>integer parameter $L$ </LI>
</UL>
and generates a new file in FASTA format containing $N$ reads composed of exactly $L$ symbols obtained by sampling the genome $R$ at random positions. For instance if $N=100000$ and $L=100$, the generator will produce a FASTA file containing 100 thousand reads from random positions in the genome, each of which is 100 nucleotides long. 
    
The FASTA headers should contain the original genomic position of each read so the mapper can verify that the correctness of the positions. For instance

&gt;R1 1,1043<br>
ACTTACTTTACTATTCATTCTACATTCTA<br>
&gt;R2 3,54654<br>
TATTTATTTCTCTTATCTATCTATCTATA<br>

is a FASTA file with two reads named R1 and R2. R1 originates from position 1043 and R2 originate from position 54654 in the genome $R$.
    
You are allowed to use Biopython to read and write FASTA files. It is mandatory to acknowledge sources.

Overview:
This is the minimapper project using a custom FM-index to search. It is not efficient but it works. It is probably worse than using a sliding window.

Limitations:
time, takes a long time to find patterns
exactness, can only find exact matches
memory, cannot get all rotations of an entire contig without running out of memory
&nbsp;    gets rotations in chunks as a workaround


Conclusions and Time Complexity:

Reader: time scales primarily with number of patterns
    1000 patterns: 35s  
    10000 patterns: ~350s
    

Mapper: time scales primarily with the number of patterns  
    1 pattern= 27s for 1.4 million bp worth of bwt = 135s for the entire bd genome with 1 pattern to match  
    +50 patterns gives +50s of runtime (50 patterns -> 79s runtime for 1.4 mil bp)  
    another +50 gives +56s of runtime (100 patterns -> 135s runtime for 1.4 mil bp)  
    +900 gives +993s of runtime (1000 patterns -> 1128s runtime for 1.4 mil bp)      

    

In [1]:

#PURPOSE:
#35s for 1000
#~350s for 10000
#~3500s ~= 1hr for 100,000
#Notes:
    #Sampled without replacement, no repeats
    #Contigs sometimes change ranges???

from Bio import SeqIO as seq
import random
from time import time

#random amongst the entire genome
#equivalent to random amongst a combination of all records in the fasta file
#create record index
#using https://biopython.org/wiki/SeqIO

start = time()
'''>NW_015449929.1 Tetranychus
urticae unplaced genomic scaffold, ASM23943v1 scaffold_223, whole genome shotgun sequence"'''

N = 1000
L = 100


samples = []

#-------------------------------------------------------------------
#Getting indexes or ranges for the contigs
#record_pos_index, the absolute cumulative length of each contig
#starts_pos_index, the absolute cumulative length of each valid start in each contig
#valid start, places where a read can start and not go out of index


#fasta_file = seq.parse("./minimapper/BD.fasta", "fasta")
#fasta_index = seq.index("./minimapper/BD.fasta", "fasta")
fasta_index = seq.index("./BD.fasta", "fasta")

start_i = 0
start_i_valid = 0
record_pos_index = [] #stores an array of genome_position,record_id pairs as an index
starts_pos_index = [] #range of valid starts

                
for record_id in fasta_index: 
    #print("index record: ",record_id)
    end_i = start_i+(len(fasta_index[record_id])-1) #99, total contig range
    record_pos_index.append((record_id,start_i))

    end_i_valid = start_i_valid+(len(fasta_index[record_id])-L-1)
    starts_pos_index.append((record_id,start_i_valid))

    start_i_valid = end_i_valid +1
    start_i = end_i+1


valid_end = start_i_valid #set the end of the last contig, exclusive
fasta_length = start_i 
print("total length: ", fasta_length)
print("total length valid:",valid_end)
#print("record_position index: ", record_pos_index)
print("cumulative contig positions: ")
for ((record_id,genome_pos),(record_id,valid_start_pos)) in zip(record_pos_index,starts_pos_index):
    print(record_id,genome_pos,valid_start_pos)



#--------------------------------------------------------------------------
#Sampling random reads and reading from the fasta_index





random_reads = (random.sample(range(valid_end),N))
#NOT SORTING RANDOM reads, defeats purpose, takes too long
random_reads.sort() #sorting, group reads by contig
#print("random reads: ",random_reads)

#position_difference, the position relative to the start of the contig
#starts_pos_index, indexes of the valid starts for each contig
def get_read(random_read, fasta_index,record_pos_index,starts_pos_index):
    
    #random_read = 7016837
    i = 0
    for record_id,index in starts_pos_index:
        if random_read-index < 0:
            break
        i+=1
    i-=1#went too far
        #read is in previous index
    position_difference = random_read-starts_pos_index[i][1]

    #print("found in start_pos index i",i)
    #print("position difference",position_difference)

    record_id = record_pos_index[i][0] #start from the absolute position
    seq = (fasta_index[record_id].seq)
    read = ">" + record_id+ ","+ str(position_difference)+'\n'+str(seq[position_difference:position_difference+L])+'\n'
    return (read)


file = open("cs144part1_1000.txt", "w") 
for read in random_reads:
    temp = get_read(read,fasta_index,record_pos_index,starts_pos_index)
    file.write(temp)
file.close()


#get a read
record_id = record_pos_index[0][0] #start from the absolute position
sequence = (fasta_index[record_id].seq)
file = open("cs144input.txt","w")
length = 10000
file.write(str(sequence[:length]))
file.close()

#first_record = repr(next(genome)
print(type(fasta_index))

print((fasta_index["contig1"]))


record_id = "contig"+str(1)

'''
for record in fasta_file:
    print(len(record)) #length of record
    end_i = start_i+(len(record)-1) #99
    record_iarr.append(end_i)
    start_i = end_i+1 #100
    
    
    print("record id ",record.id) #id of record
    print("record desc",record.description)
    print(repr(record.seq)) #sequence of record, small snippet
    #print("100,000th pos ", str(record.seq)[100000:100010])
    #print(dir(type(record))) #functions in the seqrecords class
'''

print("execution time: ", time()-start)

total length:  7362482
total length valid: 7361582
cumulative contig positions: 
contig1 0 0
contig2 1434593 1434493
contig3 2502290 2502090
contig4 3449599 3449299
contig5 4311925 4311525
contig6 5170737 5170237
contig7 5935667 5935067
contig8 6576258 6575558
contig9 7017637 7016837
<class 'Bio.File._IndexedSeqFileDict'>
ID: contig1
Name: contig1
Description: contig1
Number of features: 0
Seq('CCAGACTTGCCCTCCAATTGATACTCTGGAAGGGGTTTGGATTCCCATCATTCC...ATG')
execution time:  34.490427017211914


<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>Mapper</h1></center>
<br>

In the second part, you will write a <B>read mapper</B>, that takes in input
<UL>
<LI>a reference genome $R$ in FASTA format</LI>
<LI>a set $S$ of reads in FASTA format (generated by the generator)</LI>
</UL>
The read mapper finds the position(s) of each read in $S$ in the genome $R$, and counts how many are mapped correctly by comparing them to the original position stored in the FASTA header. You can use the original position stored in the FASTA headers <B>only</B> to verify the correctness. 

You are allowed to use any Python package that implements any of the data structures we saw in class (suffix arrays, suffix trees, FM-index (BWT), or hash tables, etc). You should be able install packages using `!pip install package`. It is mandatory to acknowledge sources.

Collect experimental results on time spent by the mapper for several choices of $|R|$ (genome size) and $N$. You can keep $L$ fixed at 100. Is the time linear in $|R|$, or is it super-linear? Is the time linear in $N$ or is it super-linear?

In [2]:
#Importing BWT from https://python.algorithms-library.com/compression/burrows_wheeler

#Can ignore this file, just for reference for BWT
#reused some of the BWT code in the block below

#Other considerations:
#bowtie and bowtie2, not compatible with windows, required linux virtual machine
#linear suffix array, ukkonen's algorithm, could not find a library online


#def all_rotations(s: str) -> list[str]:
def all_rotations(s: str):
    """
    :param s: The string that will be rotated len(s) times.
    :return: A list with the rotations.
    :raises TypeError: If s is not an instance of str.
    Examples:

    >>> all_rotations("^BANANA|") # doctest: +NORMALIZE_WHITESPACE
    ['^BANANA|', 'BANANA|^', 'ANANA|^B', 'NANA|^BA', 'ANA|^BAN', 'NA|^BANA',
    'A|^BANAN', '|^BANANA']
    >>> all_rotations("a_asa_da_casa") # doctest: +NORMALIZE_WHITESPACE
    ['a_asa_da_casa', '_asa_da_casaa', 'asa_da_casaa_', 'sa_da_casaa_a',
    'a_da_casaa_as', '_da_casaa_asa', 'da_casaa_asa_', 'a_casaa_asa_d',
    '_casaa_asa_da', 'casaa_asa_da_', 'asaa_asa_da_c', 'saa_asa_da_ca',
    'aa_asa_da_cas']
    >>> all_rotations("panamabanana") # doctest: +NORMALIZE_WHITESPACE
    ['panamabanana', 'anamabananap', 'namabananapa', 'amabananapan',
    'mabananapana', 'abananapanam', 'bananapanama', 'ananapanamab',
    'nanapanamaba', 'anapanamaban', 'napanamabana', 'apanamabanan']
    >>> all_rotations(5)
    Traceback (most recent call last):
        ...
    TypeError: The parameter s type must be str.
    """
    if not isinstance(s, str):
        raise TypeError("The parameter s type must be str.")

    return [s[i:] + s[:i] for i in range(len(s))]

#print(all_rotations("ATTACA"))


def bwt_transform(s: str) -> dict:
    """
    :param s: The string that will be used at bwt algorithm
    :return: the string composed of the last char of each row of the ordered
    rotations and the index of the original string at ordered rotations list
    :raises TypeError: If the s parameter type is not str
    :raises ValueError: If the s parameter is empty
    Examples:

    >>> bwt_transform("^BANANA")
    {'bwt_string': 'BNN^AAA', 'idx_original_string': 6}
    >>> bwt_transform("a_asa_da_casa")
    {'bwt_string': 'aaaadss_c__aa', 'idx_original_string': 3}
    >>> bwt_transform("panamabanana")
    {'bwt_string': 'mnpbnnaaaaaa', 'idx_original_string': 11}
    >>> bwt_transform(4)
    Traceback (most recent call last):
        ...
    TypeError: The parameter s type must be str.
    >>> bwt_transform('')
    Traceback (most recent call last):
        ...
    ValueError: The parameter s must not be empty.
    """
    if not isinstance(s, str):
        raise TypeError("The parameter s type must be str.")
    if not s:
        raise ValueError("The parameter s must not be empty.")

    rotations = all_rotations(s)
    rotations.sort()  # sort the list of rotations in alphabetically order
    # make a string composed of the last char of each rotation
    return {
        "bwt_string": "".join([word[-1] for word in rotations]),
        "idx_original_string": rotations.index(s),
        "bwt_firstcol":"".join([word[0] for word in rotations])
    }


#test_str = "abracadabra$"
#print(bwt_transform(test_str))


def reverse_bwt(bwt_string: str, idx_original_string: int) -> str:
    """
    :param bwt_string: The string returned from bwt algorithm execution
    :param idx_original_string: A 0-based index of the string that was used to
    generate bwt_string at ordered rotations list
    :return: The string used to generate bwt_string when bwt was executed
    :raises TypeError: If the bwt_string parameter type is not str
    :raises ValueError: If the bwt_string parameter is empty
    :raises TypeError: If the idx_original_string type is not int or if not
    possible to cast it to int
    :raises ValueError: If the idx_original_string value is lower than 0 or
    greater than len(bwt_string) - 1

    >>> reverse_bwt("BNN^AAA", 6)
    '^BANANA'
    >>> reverse_bwt("aaaadss_c__aa", 3)
    'a_asa_da_casa'
    >>> reverse_bwt("mnpbnnaaaaaa", 11)
    'panamabanana'
    >>> reverse_bwt(4, 11)
    Traceback (most recent call last):
        ...
    TypeError: The parameter bwt_string type must be str.
    >>> reverse_bwt("", 11)
    Traceback (most recent call last):
        ...
    ValueError: The parameter bwt_string must not be empty.
    >>> reverse_bwt("mnpbnnaaaaaa", "asd") # doctest: +NORMALIZE_WHITESPACE
    Traceback (most recent call last):
        ...
    TypeError: The parameter idx_original_string type must be int or passive
    of cast to int.
    >>> reverse_bwt("mnpbnnaaaaaa", -1)
    Traceback (most recent call last):
        ...
    ValueError: The parameter idx_original_string must not be lower than 0.
    >>> reverse_bwt("mnpbnnaaaaaa", 12) # doctest: +NORMALIZE_WHITESPACE
    Traceback (most recent call last):
        ...
    ValueError: The parameter idx_original_string must be lower than
    len(bwt_string).
    >>> reverse_bwt("mnpbnnaaaaaa", 11.0)
    'panamabanana'
    >>> reverse_bwt("mnpbnnaaaaaa", 11.4)
    'panamabanana'
    """
    if not isinstance(bwt_string, str):
        raise TypeError("The parameter bwt_string type must be str.")
    if not bwt_string:
        raise ValueError("The parameter bwt_string must not be empty.")
    try:
        idx_original_string = int(idx_original_string)
    except ValueError:
        raise TypeError(
            "The parameter idx_original_string type must be int or passive"
            " of cast to int."
        )
    if idx_original_string < 0:
        raise ValueError("The parameter idx_original_string must not be lower than 0.")
    if idx_original_string >= len(bwt_string):
        raise ValueError(
            "The parameter idx_original_string must be lower than" " len(bwt_string)."
        )

    ordered_rotations = [""] * len(bwt_string)
    for x in range(len(bwt_string)):
        for i in range(len(bwt_string)):
            ordered_rotations[i] = bwt_string[i] + ordered_rotations[i]
        ordered_rotations.sort()
    return ordered_rotations[idx_original_string]

My attempt at FM_index<br>  
naiive, store BWT->original string L mapping every index instead of every N indexes  <br>


implements traceback<br>  
definitely not efficient:<br>  
<t>    calculates bwt based on rotations, can do it faster<br> 
    big size complexity, several arrays that scale with size N<br> 
    stores all rotations in memory O(n^2) at the start<br>  
        bwt string, size n<br>  
        occurrences array, size 4n<br>  
        counts array, size 4<br>  
    big time complexity,<br>  
        getting rotations, O(n)<br>  
        have to sort rotations, quicksort, O(n log n)<br>  
        populating occurrences, counts array, O(n)<br>  
        
Functions:<br>
    all_rotations_sa()<\br><br>
    gets all the rotations of the string L<br>
    bwt_sa()<br>
        calls all_rotations_sa()
        uses the rotations to a dictionary with 2 objects:
            'bwt_string', the bwt transform in string form
            'bwt_pos', bwt->original string mapping, as a string of integers
    all_rotations_fm()
        constructs the actual fm_index<br>
        calls bwt_sa() to get the bwt<br>
        returns a tuple:<br>
            test_str_bwt, the bwt dictionary in bwt_sa()<br>
            occurrences, 5xn matrix containing the cumulative occurrences of each symbol<br>
                used in backtrack()<br>
            lexically_smaller, a dictionary with each value being the number of symbols lexically smaller<br>
                used in backtrack()<br>
            counts, the counts of each symbol<br>
                used in find_pattern()<br>

    backtrack()
        given a pos in the bwt, finds the symbol previous
        returns previous_pos,previous_symbol
            the position and symbol of the previous element in the bwt
    find_pattern()
        given a pattern and a bwt, find all found positions of the pattern in the bwt
        algorithm:
            initializes an array of potential matches, iterate through the bwt to find the last char in the pattern
            for each potential match:
                call back_track() until you get a match
                if you get a match, append it to an array
                else break and go to the next match
            returns all the matches

In [3]:
#My attempt at FM_index
#naiive, store BWT->original string L mapping every index instead of every N indexes
#implements traceback
#definitely not efficient:
    #calculates bwt based on rotations, can do it faster
    #big size complexity, several arrays that scale with size N
        #stores all rotations in memory O(n^2) at the start
        #bwt string, size n
        #occurrences array, size 4n
        #counts array, size 4
    #big time complexity,
        #getting rotations, O(n)
        #have to sort rotations, quicksort, O(n log n)
        #populating occurrences, counts array, O(n)
        #


'''References
https://en.wikipedia.org/wiki/FM-index
http://bowtie-bio.sourceforge.net/index.shtml
https://python.algorithms-library.com/compression/burrows_wheeler, for the above bwt (inefficient rotations version)
'''

#constructing fm index
#constructing C(c) lexically smaller table, counts the number of characters that are lexically smaller
    # " for each character c in the alphabet, 
    # contains the number of occurrences of 
    # lexically smaller characters in the text."
    # in this case, just counting the occurences of each base

#constructing Occ(c, k) occurrence table,
    #

#constructing both tables while using modified rotations function

A = 0
C = 1
G = 2
T = 3

#gets all the rotations of the string
#the first rotation ending with $ is always mapped to the end
def all_rotations_sa(s:str):
    ret = [((s[0:]+s[:0]),len(s)-1)] #mapped to the end
    for i in range(1,len(s)):
        ret.append(((s[i:] + s[:i]),i-1))
    #print(ret)
    return ret
    #return [((s[i:] + s[:i]),i) for i in range(start=1,stop=len(s))]


#calculates bwt of the s using rotations
def bwt_sa(s:str):
    rotations = all_rotations_sa(s)
    rotations.sort()  # sort the list of rotations in alphabetically order
    # make a string composed of the last char of each rotation

    bwt_string = ""
    positions = []
    for word,pos in rotations:
        bwt_string+= (word[-1])
        positions.append(pos)

    return { #"idx_original_string": rotations.index(s),
        "bwt_string": bwt_string,
        
        "bwt_pos":positions
    }

#calculate fm index
def all_rotations_fm(l:str ):
    if not isinstance(l, str):
        raise TypeError("The parameter s type must be str.")

    len_l = len(l)
    rotations = []
    counts = {'$':0,'A':0,'C':0,'G':0,'T':0}
    occurrences = {'$':[0]*len_l,'A':[0]*len_l,'C':[0]*len_l,'G':[0]*len_l,'T':[0]*len_l,}

    
    
    
    
    #print(all_rotations(test_str))
    
    test_str_bwt = (bwt_sa(l))
    

    #calculate rotations and get the bwt
    s = test_str_bwt['bwt_string']

    s_map = []#mapping bwt to the original string s

    #iterate through the bwt
    for i,symbol in zip(range(len_l),s):
        rotations.append(s[i:]+s[:i])
        counts[symbol]+=1


        occurrences['A'][i] = occurrences['A'][i-1]
        occurrences['C'][i] = occurrences['C'][i-1]
        occurrences['G'][i] = occurrences['G'][i-1]
        occurrences['T'][i] = occurrences['T'][i-1]
        occurrences['$'][i] = occurrences['$'][i-1]
        occurrences[symbol][i]+=1
        #print(occurrences[symbol][i])

    #print(counts)


    lexically_smaller = {'$':0,'A':0,'C':0,'G':0,'T':0}
    lexically_smaller['A'] = counts['$']
    lexically_smaller['C'] = counts['A'] + lexically_smaller['A']
    lexically_smaller['G'] = counts['C'] + lexically_smaller['C']
    lexically_smaller['T'] = counts['G'] + lexically_smaller['G']

    #print(occurrences)
    #print(lexically_smaller)

    return test_str_bwt,occurrences,lexically_smaller,counts
    #return [s[i:] + s[:i] for i in range(len(s))]

fm_index = all_rotations_fm("AAACGT$")

#last to first mapping?
#ideally linear time, realistically dont know
LEXICALLY_SMALLER = 2
OCCURRENCES = 1
def backtrack(fm_index,pos):
    #returns the symbol before in the sequence
     #index starts at 1

    bwt_transform  = fm_index[0]['bwt_string']
    #print("bwt_transform,",bwt_transform)

    symbol_at_pos = bwt_transform[pos]
    

    #lexically smaller and occurrences
    previous_pos = fm_index[LEXICALLY_SMALLER][symbol_at_pos]+fm_index[OCCURRENCES][symbol_at_pos][pos]-1
    previous_symbol = bwt_transform[previous_pos]
    #print(symbol_at_pos, pos,"|previous:",previous_symbol,previous_pos)

    return previous_symbol,previous_pos
backtrack(fm_index,0)


'''
count_keys = iter(fm_index[2])
for key in count_keys:
    print(key)'''

BWT = 0
COUNTS = 3

SYMBOL = 0
POS = 1
def find_pattern(pattern,fm_index):
    
    last_char = pattern[-1]


    num_potential_matches= fm_index[COUNTS][last_char]
    #print("number of potential matches:" ,num_potential_matches)
    
    match_arr = [0] * num_potential_matches
    target_char = last_char
    i = 0
    match_no = 0

    #initialize match_arr
    for symbol in fm_index[BWT]['bwt_string']:
        if(symbol == target_char):
            match_arr[match_no] = i
            match_no+=1

        i+=1
    pattern = pattern[:-1]
    #print("pattern",pattern)

    #print("match_arr:",match_arr)

    #find rest of matches
    #get first match

    #print('------------')
    
    len_pattern = len(pattern)

    match_no = 0
    target_pattern = pattern
    found_patterns = []
    for match in match_arr:
        for i in range(len_pattern):
            #print(pattern)
            target_char = pattern[-1]
            prev_char,pos = backtrack(fm_index,match)
            if prev_char != target_char:
                #print("no match")
                match = None
                break
            match = pos
            pattern = pattern[:-1]
        if match:
            #print("match: pos+i",match,len_pattern,match+len_pattern)

            found_patterns.append(match) #start of pattern
            #found_patterns.append(match+len_pattern) #end of pattern
            

        #reset variables
        pattern = target_pattern


        #pattern = pattern[:-1]
    #print("found patterns in bwt: ", found_patterns)
    return found_patterns


#bwt->original string l mapping, stored as bwt_pos, initialized in all_positions_sa()
class bwt_naiive:
    def __init__(self,contig):
        self.fm_index = all_rotations_fm(contig)

    #given bwt_pos, find l_pos
    def locate(self,pos):
        return self.fm_index[0]['bwt_pos'][pos]


#testing!
'''
l = "AAACCCGGGTTT$"
naiive_test = bwt_naiive(l)
print("l: ",l)
print("bwt_str: ", naiive_test.fm_index[0]['bwt_string'])
print("bwt_pos",naiive_test.fm_index[0]['bwt_pos'])
print("pos of $",naiive_test.locate(1))
print("all rotations: ",all_rotations_sa(l))
    


pos = find_pattern("AAA",naiive_test.fm_index)[0]

pos = 2

#testing bwt->l mapping
#for pos in range(len(l)):
l_pos = naiive_test.locate(pos)
print("pos in bwt",pos,naiive_test.fm_index[0]['bwt_string'][pos])
print("pos in l",l_pos,l[l_pos])
'''
a = 1
    


Driver Function
Working around memory limitations!

Problem:
naiive BWT takes n^2 memory to store and sort initially

Solution:
break the genome into reasonable chunks, 10,000 bp per chunk
construct the bwt and do all pattern matches on that chunk's bwt
repeat until entire genome is covered
Cheat and sort the reads?

In [6]:
#parse input reads

reads = []
with open("cs144part1_1000.txt",'r') as file:
    
    while True:
        header = file.readline()
        if not header:
            break
        header = header.split(',')
        contig = header[0][1:]
        pos = int(header[1][:-1])
        
        #contig = contig[:-1]#cull the newline
        
        read = file.readline()
        read = read[:-1] #cull the newline
        reads.append((contig,pos,read))
print(reads[0])
print(reads[1])
#reads = reads[:50]

('contig1', 1987, 'CACCTTGGAGGATTTGCCGGTTTGAGATCCTTGCATGGTCATCTGTAAAAAATTCACTACTAGCATTATCGCTAACGTCTCAAATAAAGTCGCCAATGGC')
('contig1', 8096, 'GGGGCCCAGAGGCTTTACTGGTCTAGGTTTTATTAAATTTCAAAATTGCCAATTCCAGACGGGAATCGTCTCGACACGATTCCCAAGGCTCTCGTTGCTT')


In [7]:
#Driver Function
class read_mapper():
    def __init__(self,l:str):
        
        self.bwt_l = bwt_naiive(l)
    def read_mapping(self,pattern:str): #look for a pattern in the contig substr
        pattern_locations = find_pattern(pattern,self.bwt_l.fm_index)
        if pattern_locations:
            for start in pattern_locations:
                found_pos = self.bwt_l.locate(start)
                #l_substr = l[found_pos:found_pos+len(pattern)]+"|"+l[found_pos+len(pattern):found_pos+len(pattern)+10]

                
                #print("found pattern at ",found_pos)
                #print("l representation:",l_substr)
                
            return pattern_locations
        else:
            return []
        

#Open fasta to read
# -----------------------------------------------------------

file = open("cs144input.txt")
input = str(file.read())

print("L:",L)
print("read length: ",len(reads))
file.close()

fasta_index = seq.index("./BD.fasta", "fasta")

record_ids = []
for record_id in fasta_index:
    record_ids.append(record_id)




#first_chunk = str(fasta_index[record_ids[0]].seq[90000:90000+chunk_length])
pattern = "AAAATGTCAATTATAAATGACATTGACAAGTACATTATAGCAACACGTCGCGATGCTACTGGAGACTATAGTAATGTTTTACCAATTGATAAAATTGGTC"
patterns = []
length_patterns = 1
for i in range(length_patterns):
    patterns.append(pattern)


#first_chunk_bwt = read_mapper(first_chunk)
#first_chunk_bwt.read_mapping(pattern)

pos = 0 #starting pos
chunk_length = 10000# ~amount of memory possible with naiive bwt, best time is about .7s to index 10,000
seq_length = len(fasta_index[record_ids[0]].seq) #search in first contig



i = 0
mapped_reads = []
correct_reads = 0
while(pos < seq_length): #search through only first contig
    #print(pos)
    next_chunk = str(fasta_index[record_ids[0]].seq[pos:pos+chunk_length]) 
    chunk_bwt = read_mapper(next_chunk)

    for contig,read_pos,pattern in reads: #read
        
        matches = chunk_bwt.read_mapping(pattern)
        for match in matches:
            chunk_pos = chunk_bwt.bwt_l.locate(match)
            chunk_pos += pos #add the current position in the contig with chunk_pos
            #print("location: ",chunk_pos)
            #print("readpos vs chunkpos: ",read_pos,chunk_pos)
            if(chunk_pos == read_pos):
                correct_reads+=1
            mapped_reads.append((contig,chunk_pos,pattern))

    pos = pos-L+chunk_length #chunk always has an additional L to catch missed sequences

print("found reads length:",len(mapped_reads))
print("number of correct reads:",correct_reads)
#print("chunk: ",first_chunk)

#get a chunk from the genome
#do read_mapping on that chunk for a particular pattern
    #maybe store the fm_index?
#>contig1,93711
#AAAATGTCAATTATAAATGACATTGACAAGTACATTATAGCAACACGTCGCGATGCTACTGGAGACTATAGTAATGTTTTACCAATTGATAAAATTGGTC
#initial




#testing
#g = "TTCATCGAACATCCGTAAAAATGGCGTTTTGTTTTTGCCGCTGCATGATGATACGAGATCGTCGCAACAATGGAATTGTGAATCGTGCATTCCCTGCCTA$"
#pattern = input[:100]
#l = input+'$
#read_mapping(l,pattern)

L: 100
read length:  1000
found reads length: 181
number of correct reads: 167


L: 100
read length:  1000
found reads length: 181
number of correct reads: 167
18m48s